In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute,IBMQ #Imports required libraries
from qiskit.tools.monitor import job_monitor as tracker #Imports a job monitor to check status of job
import datetime as dt #Imports datetime for checking time interval betweeen jobs execution and completion.

IBMQ.enable_account('token') #Loads IBMQ token

In [ ]:
n = int(8192) #Shots count between 1 to 8192
bit = int(input("Please enter number of bits: "))/5 
bit1 = int(bit)
counter = bit*5
lim = n*5
runtime = int(counter%lim + 1)
print('\n')
LoopNumber = 1 #Counter for iteration
for i in range (0,runtime):

    print('Loop Number: ', LoopNumber, '\n') #Shows the iteration number

    provider = IBMQ.get_provider(hub='ibm-q') #Connects to provider

    backendname=[] #makes a list of server names
    backendjobs=[] #makes a list of server jobs
    backendqbits=[] #makes a list of qubit counts of servers

    for backend in provider.backends():
        try:
            qubit_count = len(backend.properties().qubits)
        except:
            qubit_count = "simulated"

        if qubit_count == 5:

            backendname.append(backend.name())
            backendjobs.append(backend.status().pending_jobs)
            backendqbits.append(qubit_count)

    tmp = min(backendjobs) #From here
    minjob = int(backendjobs.index(tmp))
    minjobserver = str(backendname[minjob])
    print(minjobserver, "has been selected as the most available server.\n") #To here it selects server with least jobs for computing

    q = QuantumRegister(qubit_count,'q') #Implement a quantum register
    c = ClassicalRegister(qubit_count,'c') #Implement a classical register
    circuit = QuantumCircuit(q,c) #Creates a quantum circuit
    circuit.h(q) #Implement a hadamard gate
    circuit.measure(q,c) # Measure quantum bit into classical bit
    backend = provider.get_backend(minjobserver) #Connects to server with least jobs queued.


    job = execute(circuit, backend, shots = n,memory = True) #Starts the job
    tracker(job) #Keeps you updated about status of the job
    print('\n')

    t = dt.datetime.now() #Initialization time of the job            
    result = job.result() #Gives corresponding results of a qobj
    counts = result.get_memory(circuit) #Gives memory readouts in list format
    t2 = dt.datetime.now() #Finish time of the job
    tdelta = t2-t #Time interval

    time_text_file = open("IBMQ-BitRate.txt", "a") #Opens a txt file to write iteration,time and bit number
    time_text_file.writelines(["Loop No: " + str(LoopNumber) + " Took " +  str(tdelta) +  " seconds and generated " + str(n*5) + " bits from " + minjobserver + "\n"])
    time_text_file.close()

    randomnumbers_text_file = open("IBMQ-Output.txt", "a") #Opens a txt file to write bits

    for i in range (0,n):
        if counter == 0.0:
            break
        for y in range (0,qubit_count):
            randomnumbers_text_file.writelines(counts[i][y]+"\n")
            counter = counter - 1
            if counter == 0.0:
                break

    randomnumbers_text_file.close()
    
    LoopNumber = LoopNumber + 1 #Increases iteration number by 1
    runtime = runtime - 1
    if counter == 0.0 or runtime == 0.0:
        break
    print('Job Finished... Initializing next loop...\n') 